In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2,970 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-20 20:32:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2023-02-20 20:32:05 (10.8 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()
     

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [6]:
# Create a DataFrames to match tables
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
dropna_df= df.dropna()
dropna_df.show()
     

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [7]:
# Recreate the vine_df from Deliverable 1
vine_df = dropna_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()
# Filter DataFrame where Total_votes count is equal to or greater than 20 
filtered_vine_df = vine_df.filter("total_votes>=20")
filtered_vine_df.show()
     

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [9]:
 #Create a DataFrame where helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_reviews_df = filtered_vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

In [10]:
# Create a DataFrame rows where a review was written as part of the Vine program (paid), vine == 'Y'.. 
vine_program_helpful_reviews_df = helpful_reviews_df.filter("vine == 'Y'")
vine_program_helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|
|R13EGJI8C8YJ4Y|          5|           19|         20|   Y|                N|
| R9SNSQTK4TJOG|          2|           11|         20|   Y|                N|
|R1A76AK493C8P8|          5|           83|         90|   Y|     

In [11]:
# Create a DataFrame rows where the review was not part of the Vine program (unpaid), vine == 'N'. 
n_vine_program_helpful_reviews_df = helpful_reviews_df.filter("vine == 'N'")
n_vine_program_helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

In [12]:
total_helful_reviews_df = helpful_reviews_df.count()
total_helful_reviews_df


28339

In [13]:
total_vine_program_helpful_reviews_df = vine_program_helpful_reviews_df.count()
total_vine_program_helpful_reviews_df

61

In [14]:
total_n_vine_program_helpful_reviews_df = n_vine_program_helpful_reviews_df.count()
total_n_vine_program_helpful_reviews_df
     

28278

In [15]:
five_star_reviews_vine_program_df = vine_program_helpful_reviews_df.filter("star_rating == '5'").count()
five_star_reviews_vine_program_df

20

In [16]:
five_star_reviews_n_vine_program_df = n_vine_program_helpful_reviews_df.filter("star_rating == '5'").count()
five_star_reviews_n_vine_program_df

15686

In [17]:
percent_five_star_vine_reviews = (five_star_reviews_vine_program_df/total_vine_program_helpful_reviews_df)*100
percent_five_star_vine_reviews

32.78688524590164

In [18]:
percent_five_star_n_vine_reviews = (five_star_reviews_n_vine_program_df/total_n_vine_program_helpful_reviews_df)*100
percent_five_star_n_vine_reviews
     

55.470683923898434